In [49]:
import duckdb
import os
import boto3
import time

In [50]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

In [51]:
catalogPath = f"s3://{minioAuth['bucket']}/catalog/dapodikMetadata.ducklake" 
minioDataVersionStoreDirectory = f"s3://{minioAuth['bucket']}/ducklake/dataDapodik/"
localTempCatalogPath = '../metadata/dapodikMetadata.ducklake'

In [52]:
con = duckdb.connect()

In [53]:
con.sql("INSTALL httpfs; LOAD httpfs; INSTALL ducklake; LOAD ducklake;")
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint'].replace('http://', '')}';")
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false; SET s3_url_style = 'path';")

In [54]:
tempDB = duckdb.connect(database=localTempCatalogPath)
tempDB.close()

if os.path.exists(localTempCatalogPath):
    print(f"✅ File Catalog sementara dibuat secara lokal: {localTempCatalogPath}")
else:
    raise Exception("Gagal membuat file catalog lokal.")

✅ File Catalog sementara dibuat secara lokal: ../metadata/dapodikMetadata.ducklake


In [55]:
s3_client = boto3.client(
    's3',
    endpoint_url = minioAuth['endpoint'],
    aws_access_key_id = minioAuth['accessKey'],
    aws_secret_access_key= minioAuth['secretKey'],
    verify = False
)

In [56]:
try:
    # Target MinIO Key: 'catalog/dapodikMetadata.ducklake'
    minio_key = catalogPath.split(f"s3://{minioAuth['bucket']}/")[1]
    s3_client.upload_file(localTempCatalogPath, minioAuth['bucket'], minio_key)
    print("✅ File Catalog berhasil diunggah ke MinIO.")
    
    # Hapus file lokal sementara
    os.remove(localTempCatalogPath)
    
except Exception as e:
    raise Exception(f"❌ GAGAL mengunggah Catalog ke MinIO dengan Boto3: {e}")

✅ File Catalog berhasil diunggah ke MinIO.


In [57]:
try:
    con.close()
    print("Koneksi lama ditutup.")
except:
    pass
    
time.sleep(1) # Tunggu 1 detik untuk memastikan sumber daya dilepas

# 2. BUAT KONEKSI BARU DAN SET PARAMETER S3 LAGI
con = duckdb.connect()

print("Mencoba ATTACH dengan koneksi bersih...")

# --- SET PARAMETER ---
con.sql("INSTALL httpfs; LOAD httpfs; INSTALL ducklake; LOAD ducklake;")
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint'].replace('http://', '')}';")
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false; SET s3_url_style = 'path';")


# 3. ATTACH (FILE SUDAH ADA, HANYA PERLU DIBACA)
try:
    con.sql(f"ATTACH 'ducklake:{catalogPath}' AS JakartaLake (DATA_PATH '{minioDataVersionStoreDirectory}');")
    con.sql("USE JakartaLake;")
    con.sql("CREATE SCHEMA IF NOT EXISTS staging;")
    print("✅ Koneksi DuckLake siap dan Catalog berhasil dimuat (Attached)!")

except Exception as e:
    print("❌ GAGAL TOTAL pada ATTACH:")
    # Jika gagal, minta user melakukan verifikasi visual
    print(e)
    print("\n💡 Aksi Lanjutan:")
    print("1. Pastikan file 'dapodikMetadata.ducklake' terlihat di MinIO Console.")
    print("2. Jika Anda menggunakan Docker atau VM, pastikan firewall tidak memblokir lalu lintas port 9000.")

Koneksi lama ditutup.
Mencoba ATTACH dengan koneksi bersih...
❌ GAGAL TOTAL pada ATTACH:
IO Error: Failed to attach DuckLake MetaData "__ducklake_metadata_JakartaLake" at path + "s3://data-dapodik-kemendikdasmen/catalog/dapodikMetadata.ducklake"Cannot open database "s3://data-dapodik-kemendikdasmen/catalog/dapodikMetadata.ducklake" in read-only mode: database does not exist

💡 Aksi Lanjutan:
1. Pastikan file 'dapodikMetadata.ducklake' terlihat di MinIO Console.
2. Jika Anda menggunakan Docker atau VM, pastikan firewall tidak memblokir lalu lintas port 9000.
